# Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import dump, load
import time
from scipy.interpolate import PchipInterpolator
import scipy
from collections import defaultdict

from tqdm import tqdm

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

from itertools import chain

import os

# Data

In [2]:
# Specify the directory path
dir_path = "../files/data/TWIN/CS/CLEANED"

# Get all .csv files
csv_files = [f for f in os.listdir(dir_path) if f.endswith(".csv")]

In [3]:
total = 0
for csv_file in tqdm(csv_files):
    df = pd.read_csv("../files/data/TWIN/CS/CLEANED/{}".format(csv_file), index_col=False)
    df = df[(df['a1/t'] <= 0.5) & (df['a2/t'] <= 0.5)]

    # Get unique combinations of the first four columns
    CC_combinations = df.iloc[:, 1:8].drop_duplicates().to_numpy()

    assert len(CC_combinations) == len(df.iloc[:, 0].drop_duplicates().to_numpy())
    
    total += len(df.iloc[:, 0].drop_duplicates().to_numpy())
    # print(len(df.iloc[:, 0].drop_duplicates().to_numpy()))

    # print("=================================")
    # print("========== {} ===============".format(csv_file))
    # print("=================================")
    # print("Different W/r values: ", np.unique(CC_combinations[:,0], axis=0))
    # print("Different a1/c1 values: ", np.unique(CC_combinations[:,1], axis=0))
    # print("Different a1/t values: ", np.unique(CC_combinations[:,2], axis=0))
    # print("Different a2/c2 values: ", np.unique(CC_combinations[:,3], axis=0))
    # print("Different a2/t values: ", np.unique(CC_combinations[:,4], axis=0))
    # print("Different r/t values: ", np.unique(CC_combinations[:,5], axis=0))
    # print("Different b/t values: ", np.unique(CC_combinations[:,6], axis=0))    

100%|██████████| 114/114 [02:01<00:00,  1.07s/it]


In [4]:
total

1091419

In [ ]:
df = pd.read_csv("../files/data/TWIN/CS/CLEANED/{}".format(csv_files[0]), index_col=False)
df = df[(df['a1/t'] <= 0.5) & (df['a2/t'] <= 0.5)]

for csv_file in csv_files[1:]:
    df_ = pd.read_csv("../files/data/TWIN/CS/CLEANED/{}".format(csv_file), index_col=False)
    df_ = df_[(df_['a1/t'] <= 0.5) & (df_['a2/t'] <= 0.5)]
    df = pd.concat([df, df_], axis=0, ignore_index=True)

df

In [ ]:
# Get unique combinations of the first four columns
CC_combinations = df.iloc[:, 1:8].drop_duplicates().to_numpy()

print(len(CC_combinations))

In [ ]:
print("Different W/r values: ", np.unique(CC_combinations[:,0], axis=0))
print("Different a1/c1 values: ", np.unique(CC_combinations[:,1], axis=0))
print("Different a1/t values: ", np.unique(CC_combinations[:,2], axis=0))
print("Different a2/c2 values: ", np.unique(CC_combinations[:,3], axis=0))
print("Different a2/t values: ", np.unique(CC_combinations[:,4], axis=0))
print("Different r/t values: ", np.unique(CC_combinations[:,5], axis=0))
print("Different b/t values: ", np.unique(CC_combinations[:,6], axis=0))

In [ ]:
df.to_csv("../files/data/FINAL_CSV/TWIN_CORNER_CRACK_CS2_QUARTER_ELLIPSE.csv", index=False)

# Data Viz

In [ ]:
df = pd.read_csv("../files/data/FINAL_CSV/CORNER_CRACK_CS2_QUARTER_ELLIPSE.csv")
df

In [ ]:
d = df.to_numpy()

num_crack = np.unique(d[:,0])

print("Total cracks: ", len(num_crack))

np.random.seed(0)
fig, axs = plt.subplots(2, 5, figsize=(30,12))
for i in range(2):
    for j in range(5):
        idx = np.random.randint(0, len(num_crack))
        data = d[idx*128:idx*128+128]
        W_R = np.unique(data[:,1])
        a_c = np.unique(data[:,2])
        a_t = np.unique(data[:,3])
        r_t = np.unique(data[:,4])
        b_t = np.unique(data[:,5])

        assert len(W_R) == 1
        assert len(a_c) == 1
        assert len(a_t) == 1
        assert len(r_t) == 1
        assert len(b_t) == 1

        axs[i,j].scatter(data[:,-4], data[:,-3], color='purple', s=10)
        axs[i,j].plot(data[:,-4], data[:,-3], label="K-T", color='purple', linestyle=":")

        axs[i,j].scatter(data[:,-4], data[:,-2], color='green', s=10)
        axs[i,j].plot(data[:,-4], data[:,-2], label="K-B", color='green', linestyle=":")

        axs[i,j].scatter(data[:,-4], data[:,-1], color='red', s=10)
        axs[i,j].plot(data[:,-4], data[:,-1], label="K-P", color='red', linestyle=":")

        axs[i,j].set_title("W/R:{} a/c:{} a/t:{} r/t:{} b/t:{}".format(W_R[0], a_c[0], a_t[0], r_t[0], b_t[0]))
        axs[i,j].set_ylabel("SIFs")
        axs[i,j].set_xlabel(r"$\phi$")
        
        if i == 0 and j == 0:
            axs[i,j].legend()

plt.show()